# 多线程与多进程

## Python多进程
多进程相当于多核处理，可以把任务平均分配给每一个核，并且让它们同时执行

In [1]:
import os
import time
from multiprocessing import Process, Pool

### 2.1、os.fork()
- **操作系统将创建具有新进程 ID 的新的子进程，复制父进程的状态（内存、环境变量等）**
- **fork 函数调用一次，将返回两次，在父进程中返回值为子进程 id，在子进程中返回值为 0**
- **只在类 Unix 系统中有效，Windows 系统中无效**

In [ ]:
pid = os.fork()
if pid == 0:
    print(
        "执行子进程，子进程pid={pid}，父进程ppid={ppid}".format(pid=os.getpid(), ppid=os.getppid())
    )
else:
    print("执行父进程，子进程pid={pid}，父进程ppid={ppid}".format(pid=pid, ppid=os.getpid()))

执行父进程，子进程pid=17021，父进程ppid=16995
执行子进程，子进程pid=17021，父进程ppid=16995

### 2.2、multiprocessing 
**通过 multiprocessing.Process(target=function_handle, name=process_name, args=()) 创建 Process 需要指定在进程中运行的函数对象，可以自定义进程名**

- **run() 自定义子类时覆写**
- **start() 开启进程**
- **join(timeout=None) 阻塞进程等待子进程完成才能结束**
- **terminate() 终止进程**
- **is_alive() 判断进程是否存活**
- **守护进程是通过设置 daemon 属性实现**

In [ ]:
def worker():
    print("子进程执行中>>> pid={0}，ppid={1}".format(os.getpid(), os.getppid()))
    time.sleep(2)
    print("子进程终止>>> pid={0}".format(os.getpid()))


class MyProcess(Process):
    def __init__(self):
        Process.__init__(self)

    def run(self):
        print("子进程开始>>> pid={0}，ppid={1}".format(os.getpid(), os.getppid()))
        time.sleep(2)
        print("子进程终止>>> pid={}".format(os.getpid()))


# 使用 Process 实例
def main_wk():
    print("主进程执行中>>> pid={0}".format(os.getpid()))
    ps = []
    # 创建子进程实例
    for i in range(2):
        p = Process(target=worker(), name="worker" + str(i), args=())
        ps.append(p)
    # 开启进程
    for i in range(2):
        ps[i].start()
    # 阻塞进程
    for i in range(2):
        ps[i].join()
    print("主进程终止")


# 使用 Process 自定义的派生类实例
def main_mp():
    print("主进程开始>>> pid={}".format(os.getpid()))
    myp = MyProcess()
    myp.start()
    myp.join()
    print("主进程终止")


# 使用进程池，提供指定数量的进程给用户使用，即当有新的请求提交到进程池中时，如果池未满，则会创建一个新的进程用来执行该请求，反之，如果池中的进程数已经达到规定最大值，那么该请求就会等待，只要池中有进程空闲下来，该请求就能得到执行
def main_pp():
    print("主进程开始执行>>> pid={}".format(os.getpid()))
    pp = Pool(5)
    for i in range(10):
        # pp.apply(worker, args=(i,))  # 同步执行
        pp.apply_async(worker, args=(i,))  # 异步执行
    # 关闭进程池，停止接受其它进程
    pp.close()
    # 阻塞进程
    pp.join()
    print("主进程终止")


if __name__ == "__main__":
    # jupyter 中可能出现 main 找不到某模块描述 python 中不会，bug暂未知
    main_wk()
    main_mp()
    main_pp()

## Python多线程
**在 Python 多线程是假的，因为调度 CPU 的操作系统与 Python 解释器之间存在一层 GIL 全局解释器锁（Python 设计之初为了数据安全做出的妥协，后来出现得其他解释器如 pypy 和 jpython 中没有 GIL），线程想要执行，必须先拿到 GIL「通行证」，由于在一个 Python 进程之中 GIL 只有一个，拿不到通行证的线程，就不允许进入 CPU 执行，因此无论 CPU 有多少个核，都只能同时执行一个线程**

- **Python 多线程对 CPU 密集型与 IO 密集型的代码执行效率是不同的，因为在执行 IO 密集型代码的过程中多线程切换的主要代价是 IO 切换而执行 CPU 密集型代码多线程切换的主要代价是时间切换，通常 IO 切换比时间切换耗时更长**

- **CPU 密集型代码（各种循环处理、计算等）由于计算工作多，每个线程都是执行满 100 Tick 后再发出 GIL 的释放与再竞争，所以和单线程跑没啥区别，线程切换反而消耗了额外的时间**

- **IO 密集型代码（文件处理、网络爬虫等频繁文件读写）多线程能够有效提升效率，单线程下 IO 操作会进入 IO 等待，造成不必要的时间浪费，而开启多线程能在线程 a 等待时，自动切换到线程 b，可以不浪费 CPU 的资源，效果类似真正的多线程，通常在 IO 上节省的时间会超过在线程切换消耗的时间，因此能提升程序的执行效率**

In [2]:
import os
import time
import threading

### 1.1 创建线程

In [2]:
def run(n):
    print("task", n)
    time.sleep(1)
    print("2s")
    time.sleep(1)
    print("1s")
    time.sleep(1)
    print("0s")
    time.sleep(1)


if __name__ == "__main__":
    # target 是要执行的函数对象名，args 是函数对应的参数，以元组的形式存在
    t1 = threading.Thread(target=run, args=("t1",))
    t2 = threading.Thread(target=run, args=("t2",))
    t1.start()
    t2.start()

task t1
task t2


2s
2s
1s1s

0s0s



### 1.2、派生线程类创建
- 继承 threading.Thread 来定义线程类，其本质是重构 Thread 类中的 run 方法


In [ ]:
class MyThread(threading.Thread):
    def __init__(self, n):
        super(MyThread, self).__init__()
        self.n = n

    # 线程启动函数 run 必须写
    def run(self):
        print("task", self.n)
        time.sleep(1)
        print("2s")
        time.sleep(1)
        print("1s")
        time.sleep(1)
        print("0s")
        time.sleep(1)


if __name__ == "__main__":
    t1 = MyThread("t1")
    t2 = MyThread("t2")
    t1.start()
    t2.start()

### 1.3、守护线程

- **使用 setDaemon(True) 把所有的子线程都变成了主线程的守护线程，因此当主线程结束后，子线程也会随之结束，主线程不等待守护线程的执行完再去关闭，因此当主线程结束后，整个程序就退出了**
- **主线程的结束意味着进程的结束，守护线程在此时就会被回收，只有其他非守护线程结束且主线程执行完毕，主线程才会关闭，进程才能结束，才会回收进程整体的资源**

In [4]:
def run(n):
    print("task", n)
    time.sleep(1)
    print("3s")
    time.sleep(1)
    print("2s")
    time.sleep(1)
    print("1s")


if __name__ == "__main__":
    t = threading.Thread(target=run, args=("t1",))
    # 通过执行结果可以看出，设置守护线程之后，当主线程结束时，子线程也将立即结束，不再执行
    t.setDaemon(True)
    print("start")
    t.start()
    print("end")

start
task t1
end


3s
2s
1s


### 1.4、阻塞线程

In [ ]:
def run(n):
    print("task", n)
    time.sleep(2)
    print("5s")
    time.sleep(2)
    print("3s")
    time.sleep(2)
    print("1s")


if __name__ == "__main__":
    t = threading.Thread(target=run, args=("t1",))
    # 把子线程设置为守护线程，必须在 start() 之前设置
    t.setDaemon(True)
    print("start")
    t.start()
    # 设置主线程等待子线程结束
    t.join()
    print("end")

### 1.5、共享全局变量
- 线程是进程的执行单元，进程是系统分配资源的最小执行单位，所以在同一个进程中的多线程是共享资源的

In [6]:
g_num = 100


def work1():
    global g_num
    for i in range(3):
        g_num += 1
    print("in work1 g_num is : %d" % g_num)


def work2():
    global g_num
    print("in work2 g_num is : %d" % g_num)


if __name__ == "__main__":
    t1 = threading.Thread(target=work1)
    t1.start()
    time.sleep(1)
    t2 = threading.Thread(target=work2)
    t2.start()

in work1 g_num is : 103
in work2 g_num is : 103


### 1.6、互斥锁
- **由于线程之间是进行随机调度，当多个线程同时修改同一条数据时可能会出现脏数据，所以出现了线程锁，即同一时刻只允许一个线程执行某些操作**
- **线程锁用于锁定资源，可以定义多个锁，像下面的代码，当需要独占某一个资源时，任何一个锁都可以锁定这个资源，就好比你用不同的锁都可以把这个相同的门锁住一样**
- **由于线程之间是进行随机调度的，如果有多个线程同时操作一个对象，如果没有很好地保护该对象，会造成程序结果的不可预期，也称为「线程不安全」为了防止上面情况的发生，就出现了互斥锁（Lock）**）

In [ ]:
def work():
    global n
    lock.acquire()
    temp = n
    time.sleep(0.1)
    n = temp - 1
    lock.release()


if __name__ == "__main__":
    lock = threading.Lock()
    n = 100
    l = []
    for i in range(100):
        p = threading.Thread(target=work)
        l.append(p)
        p.start()
    for p in l:
        p.join()

### 1.7、递归锁

- **RLock 类代表可重入锁（Reentrant Lock）法和Lock类一模一样，但它支持嵌套，在同一个线程中可以进行多次锁定，也可以多次释放。如果使用 RLock，那么 acquire() 和 release() 方法必须成对出现。如果调用了 n 次 acquire() 加锁，RLock 对象会维持一个计数器来追踪 acquire() 方法的嵌套调用，必须调用 n 次 release() 才能释放锁**


In [ ]:
def func(lock):
    global gl_num
    lock.acquire()
    gl_num += 1
    time.sleep(1)
    print(gl_num)
    lock.release()


if __name__ == "__main__":
    gl_num = 0
    lock = threading.RLock()
    for i in range(10):
        t = threading.Thread(target=func, args=(lock,))
        t.start()

### 1.8、信号量

- **互斥锁同时只允许一个线程更改数据，而 BoundedSemaphore 是同时允许一定数量的线程更改数据，比如浴室有 3 个位置，那最多只允许 3 个人同时洗漱，后来的人只能等里头的人结束再进去**


In [ ]:
def run(n, semaphore):
    # 加锁
    semaphore.acquire()
    time.sleep(3)
    print("run the thread:%s\n" % n)
    # 释放
    semaphore.release()


if __name__ == "__main__":
    num = 0
    # 最多允许 5 个线程同时运行
    semaphore = threading.BoundedSemaphore(5)
    for i in range(22):
        t = threading.Thread(target=run, args=("t-%s" % i, semaphore))
        t.start()
    while threading.active_count() != 1:
        pass
    else:
        print("----------all threads done-----------")

### 1.9、线程事件

**事件 Event 是一个简单的线程同步对象，定义了一个全局 flag 用于主线程控制其他线程的执行**

- **event.clear() 将 flag 设置为 False**
- **event.set() 将 flag 设置为 True**
- **event.is_set() 判断 flag 是否为 True**
- **event.wait() 会一直监听 flag，如果没有检测到 flag 值为 False 就一直处于阻塞状态，当 flag 值为 True 便不再阻塞**


In [ ]:
event = threading.Event()


def lighter():
    count = 0
    # 初始，设置为绿灯
    event.set()
    while True:
        if 5 < count <= 10:
            # 红灯，清除标志位
            event.clear()
            print("\33 red light is on...\033")
        elif count > 10:
            # 绿灯，设置标志位
            event.set()
            count = 0
        else:
            print("\33 green light is on...\033")

        time.sleep(1)
        count += 1


def car(name):
    while True:
        # 判断是否设置了标志位
        if event.is_set():
            print("[%s] running....." % name)
            time.sleep(1)
        else:
            print("[%s] sees red light，waiting..." % name)
            event.wait()
            print("[%s] green light is on，start going..." % name)


startTime = time.time()
light = threading.Thread(
    target=lighter,
)
light.start()

car = threading.Thread(target=car, args=("MINT",))
car.start()
endTime = time.time()
print("用时：", endTime - startTime)